In [1]:
import nest_asyncio

nest_asyncio.apply()
# 1. AGENT LEVEL
import os
from agents import Agent, OpenAIChatCompletionsModel, AsyncOpenAI, Runner
import asyncio
try:

    GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
    if not GEMINI_API_KEY:
        raise ValueError("GEMINI ApI key not found")
  
    external_client = AsyncOpenAI(
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        api_key=GEMINI_API_KEY,
    )
    model = OpenAIChatCompletionsModel(
        model="gemini-2.0-flash", openai_client=external_client
    )
except Exception as e:
    print("Error", e)

In [ ]:
import asyncio
import os 
from pydantic import BaseModel
from agents import Agent, Runner, set_tracing_disabled, RunContextWrapper, function_tool
from agents.run import AgentRunner, set_default_agent_runner



class UserContext(BaseModel):
    user_id: str
    subscription_tier: str = "free"   
    has_permission: bool = False


def premium_feature_enabled(context: RunContextWrapper, agent: Agent) -> bool:
    print(f"premium_feature_enabled()")
    print(context.context.subscription_tier, context.context.subscription_tier in ["premium", "enterprise"])
    return context.context.subscription_tier in ["premium", "enterprise"]

@function_tool(is_enabled=premium_feature_enabled)
def get_weather(city: str) -> str:
    print(f"[ADV] get_weather()")
    return "Weather is sunny"

agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model=model,
    tools=[get_weather]
)

async def main():
    context = UserContext(user_id="123", subscription_tier="premium", has_permission=True)
    # context = UserContext(user_id="123", subscription_tier="basic", has_permission=True)

    result = await Runner.run(
        agent,
        "Call the get_weather tool with city 'London'",
        context=context,
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())
    

premium_feature_enabled()
premium True
[ADV] get_weather()
premium_feature_enabled()
premium True
London town is bright,
Sunny skies are shining now,
Day is clear and warm.



OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [3]:
import asyncio
import os 
from pydantic import BaseModel
from agents import Agent, Runner, set_tracing_disabled, RunContextWrapper, function_tool
from agents.run import AgentRunner, set_default_agent_runner



class UserContext(BaseModel):
    user_id: str
    subscription_tier: str = "free"   
    has_permission: bool = False


def premium_feature_enabled(context: RunContextWrapper, agent: Agent) -> bool:
    print(f"premium_feature_enabled()")
    print(context.context.subscription_tier, context.context.subscription_tier in ["premium", "enterprise"])
    return context.context.subscription_tier in ["premium", "enterprise"]

@function_tool(is_enabled=premium_feature_enabled)
def get_weather(city: str) -> str:
    print(f"[ADV] get_weather()")
    return "Weather is sunny"

agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model=model,
    tools=[get_weather]
)

async def main():
    # context = UserContext(user_id="123", subscription_tier="premium", has_permission=True)
    context = UserContext(user_id="123", subscription_tier="basic", has_permission=True)

    result = await Runner.run(
        agent,
        "Call the get_weather tool with city 'London'",
        context=context,
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())
    

premium_feature_enabled()
basic False
Okay, I will do.
Calling get_weather tool now.
City is London.



In [4]:
from agents import handoff
class UserContext(BaseModel):
    user_id: str
    subscription_tier: str = "free"  # free, premium, enterprise
    has_permission: bool = False


# This agent will use the custom LLM provider
agent = Agent(
    name="Assistant",
    instructions="You only respond for the user's request and delegate to the expert agent if needed.",
    model=model,
)

expert_agent = Agent(
    name="Expert",
    instructions="You are an expert in the field of recursion in programming.",
    model=model,
)


agent.handoffs = [handoff(expert_agent, is_enabled=lambda ctx, agent: ctx.context.has_permission)]

async def main():
    context = UserContext(user_id="123", subscription_tier="premium", has_permission=False)

    result = await Runner.run(
        agent,
        "Call the expert agent and ask about recursion in programming",
        context=context,
        max_turns=1
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())
    

```tool_code
print(agent_manager.run(user_prompt="recursion in programming"))
```


In [5]:
from agents import handoff
class UserContext(BaseModel):
    user_id: str
    subscription_tier: str = "free"  # free, premium, enterprise
    has_permission: bool = False


# This agent will use the custom LLM provider
agent = Agent(
    name="Assistant",
    instructions="You only respond for the user's request and delegate to the expert agent if needed.",
    model=model,
)

expert_agent = Agent(
    name="Expert",
    instructions="You are an expert in the field of recursion in programming.",
    model=model,
)


agent.handoffs = [handoff(expert_agent, is_enabled=lambda ctx, agent: ctx.context.has_permission)]

async def main():
    context = UserContext(user_id="123", subscription_tier="premium", has_permission=True)

    result = await Runner.run(
        agent,
        "Call the expert agent and ask about recursion in programming",
        context=context,
        max_turns=1
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())
    

MaxTurnsExceeded: Max turns (1) exceeded

In [6]:
from agents import handoff
class UserContext(BaseModel):
    user_id: str
    subscription_tier: str = "free"  # free, premium, enterprise
    has_permission: bool = False


# This agent will use the custom LLM provider
agent = Agent(
    name="Assistant",
    instructions="You only respond for the user's request and delegate to the expert agent if needed.",
    model=model,
)

expert_agent = Agent(
    name="Expert",
    instructions="You are an expert in the field of recursion in programming.",
    model=model,
)


agent.handoffs = [handoff(expert_agent, is_enabled=lambda ctx, agent: ctx.context.has_permission)]

async def main():
    context = UserContext(user_id="123", subscription_tier="premium", has_permission=True)

    result = await Runner.run(
        agent,
        "Call the expert agent and ask about recursion in programming",
        context=context,
        max_turns=2
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())
    

Alright, you've reached an expert on recursion in programming. Ask me anything! I can help you with understanding the core concepts, exploring different types of recursion, analyzing its advantages and disadvantages, comparing it to iterative approaches, and even debugging recursive code. Let me know what's on your mind.

